# 3. 타원곡선 암호

## 1. 유한체에서 정의된 타원곡선   
 : 비트코인에서 사용하는 타원곡선은 유한체에서 정의된다.
  * 유한체는 실수가 아니므로 그래프의 모양은 점이 산재된 형태를 갖는다.
  * 점 덧셈 방정식의 근의 어떤 체에 속해있는지와 무관하게 점 덧셈이 성립한다.
  * 유한체의 사칙연산으로 계산할 수 있다.

In [1]:
from ecc import FieldElement, Point

a = FieldElement(num=0, prime=223)
b = FieldElement(num=7, prime=223)
x = FieldElement(num=192, prime=223)
y = FieldElement(num=105, prime=223)
p1 = Point(x, y, a, b)
print(p1)

Point(192,105)_0_7 FieldElement(223)


### 1.2 유한체에서 정의된 타원곡선 위 두 점의 덧셈   
 : FieldElement 클래스에 유한체 원소에 대한 연산을 정의했으므로, Point 클래스를 생성할 때 FieldElement로 초기화하면 유한체에서의 점 덧셈을 실행할 수 있다.

In [2]:
prime = 223
a = FieldElement(num=0, prime=prime)
b = FieldElement(num=7, prime=prime)
x1 = FieldElement(num=192, prime=prime)
y1 = FieldElement(num=105, prime=prime)
x2 = FieldElement(num=17, prime=prime)
y2 = FieldElement(num=56, prime=prime)
p1 = Point(x1, y1, a, b)
p2 = Point(x2, y2, a, b)
print(p1+p2)

Point(170,142)_0_7 FieldElement(223)


In [6]:
prime = 223
a = FieldElement(num=0, prime=prime)
b = FieldElement(num=7, prime=prime)
x1 = FieldElement(num=143, prime=prime)
y1 = FieldElement(num=98, prime=prime)
x2 = FieldElement(num=76, prime=prime)
y2 = FieldElement(num=66, prime=prime)
p1 = Point(x1, y1, a, b)
p2 = Point(x2, y2, a, b)
print(p1+p2)

Point(47,71)_0_7 FieldElement(223)


### 1.3 타원곡선  위 정의된 스칼라 곱셈  
 : 점 덧셈은 결합법칙이 성립하므로 스칼라 곱셈을 사용할 수 있음.    
  * 스칼라 곱셈 결과는 비선형적이므로 예측하기가 굉장히 어렵다.
  * 스칼라 곱셈은 어렵지 않지만 반대 계산인 나눗셈 은 그렇지 않음.
  * 이런 문제를 '이산 로그 문제'라고 하고, 타원곡선 암호 방법의 원리가 됨.
  * 군(Group) : 스칼라 곱셈의 다른 성질은 스칼라 값을 계속 증가시켜 곱하다보면 무한원점에 도달하게 되는데, 그러면 군 집합을 얻을 수 있음.
   => {G, 2G, 3G, 4G, ..., nG} 여기서 nG = 0(무한원점)
  * 타원곡선 암호에 필요한 바탕 : 곱셈은 쉽지만 역산은 그렇지 않은 점과 유한군에서 덧셈 연산이 잘 지원되는 점
  * s * (47, 41) = (194, 172) 인 경우 군의 원소 수가 적어 s를 쉽게 구할 수 있지만, 원소 수가 늘어나면 이산 로그 문제를 다루는 것은 불가능.

### 1.4 스칼라 곱셈으로 생성된 군의 성질    
 * 유한순환군 : 실제 공개키 암호에 쓰이는 개념으로, 유한체에서 정의된 타원곡선 위 한 점에 스칼라 값을 곱해서 생성할 수 있고, 이 한점을 생성점(generator point)라고 함.
 * 체와 다르게 군에는 한 가지 연산(점 덧셈)만이 존재.
 * 스칼라 곱셈으로 생성된 군의 성질
  1) 항등원이 존재 : 항등원 = 무한원점으로 정의됨.
  2) 점 덧셈 연산에 대해 닫혀있음.
  3) 역원이 존재 : nG가 군에 속하면 (n-a)G도 군에 속한다.
  4) 교환법칙이 성립
  5) 결합법칙이 성립

In [ ]:
# Point 클래스 : 스칼라 곱셈은 int형 * Point 클래스형 객체의 곱셈 연산
def __rmul__(self, coefficient):
    product = self.__class__(None, None, self.a, self.b) # 무한 원점으로 초기화
    for i in range(coefficient):
        product += self
    return product

In [8]:
#  coefficeint가 큰 경우를 이진수 전개를 이용해 log2(n)번만에 루프를 끝내도록 개선한 코드
def __rmul__(self, coefficient):
    coef = coefficient
    current = self # 시작한느 점으로 초기화
    result = self.__class__(None, None, self.a, self.b) 
    while coef:
        if coef & 1: # 가장 오른쪽 비트가 1이라면 해당 current 값을 나중에 반환할 result에 더함
            result += current
        current += current # 현재 점 current를 2배로 만듦
        coef >>= 1  # coefficient를 오른쪽으로 비트 이동하여 최하위 비트를 탈락시키고 다음 최하위 비트 조사를 준비
    return result

## 2. 비트코인에서 사용하는 타원곡선

 : 큰 소수를 선택해 군의 원소가 많아지면 타원곡선 암호의 보안성이 높아진다.   
* 공개키 암호를 위한 타원곡선 매개변수 정의
  1) 타원곡선의 a, b   
  2) 유한체의 위수인 소수 p    
  3) 생성점 G의 x, y 좌표값    
  4) G로 생성한 군의 위수 n    
* 비트코인 타원곡선 secp256k1의 매개변수 값
  1) a = 0, b = 7, y^2 = x^3 + 6    
  2) p = 2^256 - 2^32 - 977    
  3) Gx = 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798    
  4) Gy = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8    
  5) n = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141    
  - 방정식이 상대적으로 간단하다.    
  - p가 2^256에 매우 가까우므로 2^256보다 작은 대부분의 숫자가 유한체를 형성하고, x, y 좌표는 각각 256비트로 표현된다.    
  - 군의 위수인 n도 2^256에 매우 가까운 값이므로 스칼라 값 또한 256비트로 표현할 수 있음.    
  - 2^256은 지구상의 원자수를 합친 것 보다 더 큰 숫자임.   

### 2.1 secp256k1의 매개변수로만 동작하는 유한체 클래스 정의

In [9]:
class S256Field(FieldElement): # FieldElement 클래스 상속

    def __init__(self, num, prime=None):
        super().__init__(num=num, prime=P)

    def __repr__(self):
        return '{:x}'.format(self.num).zfill(64) # 256비트 자리를 차지하도록

### 2.2 secp256k1의 점에 대한 클래스 S256Point 정의

In [11]:
class S256Point(Point): # Point 클래스 상속

    def __init__(self, x, y, a=None, b=None):
        a, b = S256Field(A), S256Field(B)
        if type(x) == int:
            super().__init__(x=S256Field(x), y=S256Field(y), a=a, b=b)
        else: # 무한원점인 경우
            super().__init__(x=x, y=y, a=a, b=b) 

    def __repr__(self):
        if self.x is None:
            return 'S256Point(infinity)'
        else:
            return 'S256Point({}, {})'.format(self.x, self.y)

    def __rmul__(self, coefficient):
        coef = coefficient % N # 군의 위수 N, nG = 0이므로 n번마다 0으로 돌아옴
        return super().__rmul__(coef)

    # 3.3 서명 검증 실습 - Signature 클래스의 인스턴스를 매개변수로 받음
    def verify(self, z, sig):
        s_inv = pow(sig.s, N - 2, N) # 군의 위수이며 소수인 N을 페르마의 작은 정리를 이용하는데 계산
        u = z * s_inv % N 
        v = sig.r * s_inv % N  
        total = u * G + v * self  
        return total.x.num == sig.r # 같은지 계산

## 3. 공개키 암호 및 서명

## 3.1 공개키 암호
$$P = eG$$
* e는 비밀키, P는 공개키에 해당
* 비대칭 방정식 : e, G를 알면 P를 계산하기 쉽지만 P와 G를 알 때 e를 계산하기 어려운 이산 로그 문제에 기반, 서명 및 검증 알고리즘의 핵심이 됨
* 비밀키는 256비트 숫자, 공개키는 x, y 각각 256비트 숫자로 구성된 좌표값 

## 3.2 서명 생성과 서명 검증
 : 서명이 필요한 곳에 비밀키로 서명(서명을 받을 도착지도 포함)을 하고, 받는 사람이 수신자의 공개키로 서명 검증을 시도해 진짜 수신자가 서명을 한 것인지 검증하는 것.
   -> 트랙잭션에서 사용된다.
### 1) 디지털 서명 알고리즘 ECDSA(타원곡선 디지털 서명 알고리즘)
$$eG = P$$
$$kG = R$$

k는 임의의 256비트 숫자를 의미한다. R의 x좌표를 r이라고 표시한다(random을 의미). (x좌표를 알면 타원곡선 방정식에 x 좌표를 대입하여 y를 구할 수 있다.)

다음 방정식은 이산 로그 문제와 동일하다.
$$uG + vG = kG$$

서명자는 k를 임의이 값으로, 그리고 u, v != 0이 아닌 값으로 선정한다. G와 P는 알려진 값이다.

u, v 가 0이 아닌 값을 선정하기 위해 e를 알고 있어 P를 eG로 대치할 수 있다면

$$eG = ((k - u)/v)G 또는 e = (k - u)/v$$

위의 식을 만족하는 모든 (u, v) 조합 중 한 가지를 선택해 선정할 수 있다.  

e를 모르는 경우 e = (k-u)/v를 만족할 때까지 (u, v) 조합을 대입해볼 수 있으나 이는 이산 로그 문제의 해를 찾는 것이므로 e를 알고 있다고 보는 것이 타당하다.    

서명에는 검증 목적을 위한 서명해시를 방정식에 포함시킨다. 서명해시는 서명하고자하는 메시지의 해시값 z를 의미한다.

$$u = z/s, v = r/s$$

u, v를 위와 같이 표현하면 v 계산에는 목표값인 r이, u 계산에는 서명해시값인 z가 들어가게 된다.

따라서 서명 알고리즘은 다음과 같다.

$$uG + vP = R = kG$$
$$uG + veG = kG$$
$$u + ve = k$$
$$z/s + re/s = k$$
$$(z + re)/s = k$$
$$s = (z + re)/k$$

서명에서 검증자에게 공개해야할 정보는 (r, s)이다.

### 2) 서명 검증
1. 서명 (r, s)와 메시지의 해시값 z, 서명자의 공개키 P
2. u = z/s, v = r/s를 계산
3. uG + vP = R을 계산
4. 만약 R의 x좌표가 r과 같다면 서명은 유효하다.    



* 서명 해시 z는 메시지에 hash256 함수(sha256 함수는 두 번 적용하는 것)를 통해 32바이트(256비트)의 z를 얻음.
* 검증하려는 것은 kG = R에서 다시 R을 구할 수 있는가 하는 것.
* R을 구하는 유일한 길은 e를 알고 있는 것이므로 v의 계산에 r을 사용함으로써 z처럼 서명자가 R도 어떤 값인지 안다는 사실을 증명함.

## 3.3 서명 검증 실습

In [1]:
class Signature: # r, s를 담을 수 있는 Signature 클래스

    def __init__(self, r, s):
        self.r = r
        self.s = s

    def __repr__(self):
        return 'Signature({:x},{:x})'.format(self.r, self.s)

## 3.4 서명 생성 절차
 1) 서명해시 z와 비밀키 e는 이미 알고 있음 (eG = P)   
 2) 임의의 k를 선택   
 3) R = kG로부터 R의 x 좌푯값 r을 계산   
 4) s = (z + re) / k를 계산   
 5) 서명은 (r,s)   

In [ ]:
class PrivateKey:

    def __init__(self, secret):
        self.secret = secret
        self.point = secret * G # 공개키에 해당하는 self.point를 계산 및 보관

    def hex(self):
        return '{:x}'.format(self.secret).zfill(64)

    def sign(self, z):
        k = self.deterministic_k(z) # 임의의 k값 생성
        r = (k * G).x.num # kG의 x 좌표값
        k_inv = pow(k, N - 2, N) # 페르마의 작은 정리 이용
        s = (z + r * self.secret) * k_inv % N # s = (z + re) / k
        if s > N / 2: 
            s = N - s
        return Signature(r, s)

    # 무작위 k값 선정 메소드 - k를 재사용하면 비밀키가 드러나게 된다.
    def deterministic_k(self, z):
        k = b'\x00' * 32
        v = b'\x01' * 32
        if z > N:
            z -= N
        z_bytes = z.to_bytes(32, 'big')
        secret_bytes = self.secret.to_bytes(32, 'big')
        s256 = hashlib.sha256
        k = hmac.new(k, v + b'\x00' + secret_bytes + z_bytes, s256).digest()
        v = hmac.new(k, v, s256).digest()
        k = hmac.new(k, v + b'\x01' + secret_bytes + z_bytes, s256).digest()
        v = hmac.new(k, v, s256).digest()
        while True:
            v = hmac.new(k, v, s256).digest()
            candidate = int.from_bytes(v, 'big')
            if candidate >= 1 and candidate < N:
                return candidate  # <2>
            k = hmac.new(k, v + b'\x00', s256).digest()
            v = hmac.new(k, v, s256).digest()